In [70]:
import numpy as np
import pandas as pd
import time
import multiprocessing as mp
print('Number of Core :',mp.cpu_count())

class VehicleRoutingArrangement:
    def __init__(self, node_names, distances_matrix, tasks):
        self.full_node_names = node_names
        self.node_names = node_names['name'].values
        self.distances = distances_matrix
        self.tasks = tasks
        
    def checkRemainingTasks(self, job_done):
        remaining_tasks = self.tasks.copy()
        if len(remaining_tasks) > 0:
            active_path = tasks['path'].values
            job_done = job_done[np.isin(job_done, active_path)]
            
            job_unique, job_counts = np.unique(job_done, return_counts=True)
            job_done_df = pd.DataFrame({'path': job_unique, 'Count': job_counts})
            
            merged_df = remaining_tasks.merge(job_done_df, on='path',how='outer')
            merged_df.loc[merged_df['Count_y'].isnull(), 'Count_y'] = 0
            merged_df['Count'] = merged_df['Count_x'] - merged_df['Count_y']
            output = merged_df[merged_df['Count'] > 0][['Start','End','path','Count']]        
            return output
        else:
            return remaining_tasks
    
    def startOrEnd(self, current_position,rm_task):
        start_set = rm_task['Start'].unique()
        if current_position in start_set:
            return rm_task[rm_task['Start'] == current_position]['End'].values
        else:
            return start_set
        
    def nextJobsMatrix(self, job_done, next_nodes):
        current_position = job_done[-1].split('-')[-1]
        next_jobs = current_position + '-' + next_nodes
        
        if len(job_done[0]) > 1:
            ini_mat = np.tile(job_done,(len(next_jobs),1))
            output = np.hstack((ini_mat, np.array([next_jobs]).T ))
        else:
            output = np.array([next_jobs]).T
        return output
    
    def jobMatrix(self, each_row):
        remaining_jobs = self.checkRemainingTasks(each_row)
        if len(remaining_jobs) == 0:
            job_matrix = np.append(each_row,'0')
        else:
            current_position = each_row[-1].split('-')[-1]
            next_nodes = self.startOrEnd(current_position, remaining_jobs)
            job_matrix = self.nextJobsMatrix(each_row, next_nodes)
        return job_matrix
    
    def currentPosition(self, x):
        return x.split('-')[-1]
    
    def getUniqueRows(self, output):
        # re-output to unique matrix
        old_output = output.copy()
        vCurrentPos = np.vectorize(self.currentPosition)
        last_position = old_output[:,-1]
        current_pos_arr = vCurrentPos(last_position)
        output.sort(axis=1)
        all_groups = pd.DataFrame({
                        'tasks' : ["".join(i) for i in output.astype(str)], 
                        'current_pos': list(current_pos_arr) , 
                        'id': range(len(current_pos_arr)) }).groupby(['tasks','current_pos'])
        unique_index = all_groups.first()['id'].values
        output = old_output[unique_index,:]
        return output 
    
    def allPath(self, output):
        # Pool Multiple Processing    
        pool = mp.Pool(processes=15)
        
        last_column = output[:,-1]
        all_zero = sum(last_column == '0')
        while all_zero != len(last_column):
            # find all path
            start_time = time.time() 
            output_list = pool.map(self.jobMatrix, output)
            output = np.vstack(output_list)
            output = self.getUniqueRows(output)
            
            last_column = output[:,-1]
            all_zero = sum(last_column == '0')
            stop_time = time.time() - start_time
            print('Output Matrix Shape :', output.shape, ', Use Time :', stop_time, 's')
            del output_list
        
        return output 
    
    def getDistance(self, startEndStr):
        tasks_arr = self.tasks['path'].values
        if startEndStr == '0':
            return 0
        else:
            start,end = startEndStr.split('-')
            i,j = np.where(self.node_names == start)[0][0],np.where(self.node_names == end)[0][0]
            return self.distances[i,j]
        
    def positiveDistance(self):
        task_list = []
        for i in range(len(self.tasks)):
            task_list = task_list + [self.tasks['path'].values[i]] *self.tasks['Count'].values[i]
        getDistanceAllElement = np.vectorize(self.getDistance) 
        result_array = getDistanceAllElement(task_list)
        return sum(result_array)
        
    def sumDistance(self, distance_arr):        
        pos_distances = self.positiveDistance()
        abs_distances = sum(distance_arr)
        neg_distances = pos_distances - abs_distances
        all_distances = pos_distances + neg_distances
        return pos_distances, neg_distances, all_distances, abs_distances
        
    def allDistancePath(self, all_path):
        getDistanceAllElement = np.vectorize(self.getDistance) 
        result_array = getDistanceAllElement(all_path)
        all_distance_path = np.apply_along_axis(self.sumDistance, 1, result_array)
        return all_distance_path
    
    def getTaskName(self, task_path):
        name_arr = self.full_node_names['name'].values
        only_task = task_path[task_path != '0']

        # แยกจุดเริ่ม-จบ 
        start_f = lambda x : x.split('-')[0]
        start_vf = np.vectorize(start_f)
        start_node = start_vf(only_task)
        end_f = lambda y : y.split('-')[1]
        end_vf = np.vectorize(end_f)
        end_node = end_vf(only_task)

        row_num = len(only_task)

        name_list = []
        distance_list = []
        for i in range(row_num):
            # name        
            start_name =  self.full_node_names[self.full_node_names['name'] == start_node[i]]['node'].values[0]
            end_name =  self.full_node_names[self.full_node_names['name'] == end_node[i]]['node'].values[0]
            name = start_name + '-' + end_name 
            name_list.append(name)

            # distance        
            start,end = start_node[i],end_node[i]
            m,n = np.where(name_arr == start)[0][0],np.where(name_arr == end)[0][0]
            before_path = only_task[0:i]
            remaining_tasks = self.checkRemainingTasks(before_path)
            current_task = task_path[i] 
            if current_task in remaining_tasks['path'].values:
                distance_list.append(self.distances[m,n])
            else:
                distance_list.append(-self.distances[m,n])

        return pd.DataFrame({'Name' : name_list, 'Distance': distance_list})
    
start_point = np.array([['S']]) # start_point
node_names = pd.read_csv('data/node_names.csv')
distances = pd.read_csv('data/distances_matrix.csv', header=None).values
tasks = pd.read_csv('data/tasks.csv')[['Start','End','Count']]
tasks['path'] = tasks['Start'] +'-'+ tasks['End']

Number of Core : 4


In [71]:
vehicleObj = VehicleRoutingArrangement(node_names, distances, tasks)

โหลด All Path จากไฟล์

In [3]:
all_path = np.load('../all_path.npy')

In [59]:
all_path.shape

(502778, 30)

In [60]:
all_path

array([['S-T', 'T-E', 'E-G', ..., 'S-T', '0', '0'],
       ['S-T', 'T-S', 'S-T', ..., 'A-Q', '0', '0'],
       ['S-T', 'T-E', 'E-G', ..., 'S-T', '0', '0'],
       ...,
       ['S-T', 'T-B', 'B-N', ..., 'T-A', 'A-B', '0'],
       ['S-T', 'T-A', 'A-P', ..., 'T-A', 'A-B', '0'],
       ['S-T', 'T-A', 'A-P', ..., 'T-A', 'A-B', '0']], dtype=object)

In [61]:
all_path[0]

array(['S-T', 'T-E', 'E-G', 'G-A', 'A-L', 'L-A', 'A-L', 'L-A', 'A-P',
       'P-D', 'D-L', 'L-A', 'A-P', 'P-E', 'E-M', 'M-A', 'A-B', 'B-P',
       'P-B', 'B-P', 'P-B', 'B-N', 'N-A', 'A-Q', 'Q-S', 'S-T', 'T-S',
       'S-T', '0', '0'], dtype=object)

In [63]:
distance_path = vehicleObj.allDistancePath(all_path) # unique อยู่แล้ว

ระยะทางที่มีของ 2438

In [64]:
distance_path

array([[ 2438.4, -2250.3,   188.1,  4688.7],
       [ 2438.4, -2092.3,   346.1,  4530.7],
       [ 2438.4, -2293.3,   145.1,  4731.7],
       ...,
       [ 2438.4, -1995.1,   443.3,  4433.5],
       [ 2438.4, -1991.3,   447.1,  4429.7],
       [ 2438.4, -1977.2,   461.2,  4415.6]])

ค่าดีสุดคือน้อยที่สุด เมื่อ sort แล้ว ค่าดีสุดจะอยู่อันแรก

In [66]:
np.sort(distance_path[:,3])

array([3568.7, 3569.6, 3569.6, ..., 4912.7, 4913.8, 4917.6])

In [67]:
path_from_number_of_distances = all_path[distance_path[:,3] == np.sort(distance_path[:,3])[0]] # เอาค่าดีสุด
path_from_number_of_distances

array([['S-T', 'T-A', 'A-P', 'P-A', 'A-P', 'P-A', 'A-B', 'B-N', 'N-E',
        'E-G', 'G-E', 'E-M', 'M-B', 'B-P', 'P-B', 'B-P', 'P-A', 'A-L',
        'L-S', 'S-T', 'T-A', 'A-L', 'L-D', 'D-L', 'L-S', 'S-T', 'T-A',
        'A-Q', '0', '0']], dtype=object)

In [72]:
thai_name = vehicleObj.getTaskName(path_from_number_of_distances[0])
thai_name

,Name,Distance
0,ลำปลายมาศ-วังแดง,322.0
1,วังแดง-นครหลวง,-11.6
2,นครหลวง-โคกตูม,45.6
3,โคกตูม-นครหลวง,-45.6
4,นครหลวง-โคกตูม,45.6
5,โคกตูม-นครหลวง,-45.6
6,นครหลวง-ท่าเรือ,19.8
7,ท่าเรือ-หนองแค(สัตว์บก),43.2
8,หนองแค(สัตว์บก)-บางปะกง,-148.0
9,บางปะกง-บางนา,30.2


In [73]:
distance_path[distance_path[:,3] == np.sort(distance_path[:,3])[0]]

array([[ 2438.4, -1130.3,  1308.1,  3568.7]])